In [7]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
from keras.backend import tensorflow_backend
from keras.layers.core import K
import copy

K.set_learning_phase(0)
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)

In [8]:
def LoadData():
    test = pd.read_csv('./data/test.csv')

    X = np.array(test['feature'])
    X_test = []
    for i in range(X.shape[0]):
        x = np.array(X[i].split(' '))
        x = x.astype(np.int)
        X_test.append(x)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0],48,48,1)/255
    
    return X_test

In [9]:
X_train = LoadData()

In [10]:
modellist = []
weight = []

# vote12
modellist += ['./model/bag/0.h5', './model/bag/8.h5', 'model/model27.h5', 
              'model/model28.h5', 'model/model30.h5', 'model/model33.h5']

weight += [ 0.67, 0.67, 0.6853, 0.68681 , 0.66, 0.679]

modellist += ['model/model31.h5']
weight += [0.672]

xss = X_test

final = np.array([0.0] * xss.shape[0] * 7).reshape(xss.shape[0],7)
outfile = "final2"

for i in range(len(modellist)):
    model = load_model(modellist[i])
    predict = model.predict(xss)
    final += predict * (weight[i])
    del model

for cnt, i in enumerate(final):
    final[cnt,:] = (i) / sum(i)
    
final[:,4] -= 0.08497144288577155
final[:,6] -= 0.08877867735470943

Y_pred = np.array([])
for i in range(len(final)):
    Y_pred = np.append(Y_pred, int(np.argmax(final[i])))

ans = []
for i in range(len(Y_pred)):
    ans.append([i,int(Y_pred[i])])
np.mean(Y_pred == Y_train)
ans = pd.DataFrame(ans,columns=['id', 'label'])
ans.to_csv('data/'+ outfile +'.csv', index=False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
